In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stores-area-and-sales-data/Stores.csv


In [2]:
data = pd.read_csv("/kaggle/input/stores-area-and-sales-data/Stores.csv")

In [3]:
from IPython.display import display

display(data)

,Store ID,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales
0,1,1659,1961,530,66490
1,2,1461,1752,210,39820
2,3,1340,1609,720,54010
3,4,1451,1748,620,53730
4,5,1770,2111,450,46620
...,...,...,...,...,...
891,892,1582,1910,1080,66390
892,893,1387,1663,850,82080
893,894,1200,1436,1060,76440
894,895,1299,1560,770,96610


In [4]:
data.drop(["Store ID "], axis = 1, inplace = True) 

for entry in data:
    print(entry)

Store_Area
Items_Available
Daily_Customer_Count
Store_Sales


In [5]:
display(data)

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales
0,1659,1961,530,66490
1,1461,1752,210,39820
2,1340,1609,720,54010
3,1451,1748,620,53730
4,1770,2111,450,46620
...,...,...,...,...
891,1582,1910,1080,66390
892,1387,1663,850,82080
893,1200,1436,1060,76440
894,1299,1560,770,96610


In [6]:
import pandas as pd
from sklearn import preprocessing

x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)

In [7]:
display(data)

,0,1,2,3
0,0.607978,0.593084,0.335484,0.508580
1,0.471802,0.472622,0.129032,0.245562
2,0.388583,0.390202,0.458065,0.385503
3,0.464924,0.470317,0.393548,0.382742
4,0.684319,0.679539,0.283871,0.312623
...,...,...,...,...
891,0.555021,0.563689,0.690323,0.507594
892,0.420908,0.421326,0.541935,0.662327
893,0.292297,0.290490,0.677419,0.606706
894,0.360385,0.361960,0.490323,0.805621


In [8]:
for column in data:
    
    arr = data[column].isnull()
    
    for entry in arr:
        
        if entry != False:
            print("NaN")
            
print("Done")

Done


In [9]:
train_data = data[0:int(len(data)*.7)]
test_data = data[int(len(data)*.7):len(data)]

In [10]:
for entry in train_data:
    print(entry, type(entry))

0 <class 'int'>
1 <class 'int'>
2 <class 'int'>
3 <class 'int'>


In [11]:
train_label = train_data[3]
train_data = train_data[[0, 1, 2]]

test_label = test_data[3]
test_data = test_data[[0, 1, 2]]

In [12]:
from IPython.display import display

display(train_label)
display(train_data)

0      0.508580
1      0.245562
2      0.385503
3      0.382742
4      0.312623
         ...   
622    0.353156
623    0.555917
624    0.544280
625    0.460256
626    0.536982
Name: 3, Length: 627, dtype: float64

,0,1,2
0,0.607978,0.593084,0.335484
1,0.471802,0.472622,0.129032
2,0.388583,0.390202,0.458065
3,0.464924,0.470317,0.393548
4,0.684319,0.679539,0.283871
...,...,...,...
622,0.409904,0.405187,0.458065
623,0.504127,0.513545,0.329032
624,0.193948,0.202882,0.477419
625,0.437414,0.431124,0.683871


In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import datetime
from tensorboard.plugins.hparams import api as hp

# %load_ext tensorboard
%reload_ext tensorboard

# rm -rf ./logs/

model = keras.Sequential()

model.add(layers.Dense(12, activation='relu', input_shape=(3,)))

model.add(layers.Dense(9, activation='relu'))

model.add(layers.Dense(1, activation='relu'))

model.summary()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['accuracy'])

model.fit(np.array(train_data), np.array(train_label), epochs=100, batch_size=30, validation_data = (np.array(test_data), np.array(test_label)), 
        callbacks=[tensorboard_callback])

score = model.evaluate(np.array(test_data), np.array(test_label), verbose=0)

print(score)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 117       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10        
Total params: 175
Trainable params: 175
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
21/21 [==============================] - 1s 16ms/step - loss: 0.2131 - accuracy: 0.0016 - val_loss: 0.2385 - val_accuracy: 0.0000e+00
Epoch 2/100
21/21 [==============================] - 0s 4ms/step - loss: 0.2131 - accuracy: 0.0016 - val_loss: 0.2385 - val_accuracy: 0.0000e+00
Epoch 3/100
21/21 [==============================] - 0s 4ms/step - loss: 0.2131 - accuracy: 0.001

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
rf_reg = RandomForestRegressor()

reg_list = [rf_reg]
for reg in reg_list:
        
    reg.fit(train_data,train_label)
        
    y_pred = reg.predict(train_data)
    
    mse = mean_squared_error(y_pred,train_label)
    rmse = np.sqrt(mean_squared_error(y_pred,train_label))
    mae = mean_absolute_error(y_pred,train_label)
    score = reg.score(train_data,train_label)
    
    print('Regressor:{}\nMSE:{:.2f}\nRMSE:{:.2f}\nMAE:{:.2f}\nScore:{:.4f}\n\n'.format(str(reg),mse,rmse,mae,score))

Regressor:RandomForestRegressor()
MSE:0.00
RMSE:0.07
MAE:0.05
Score:0.8381


